In [2]:
# Imports
import anfis
import os
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from pyfume.Clustering import Clusterer
from pyfume.EstimateAntecendentSet import AntecedentEstimator
from pyfume.EstimateConsequentParameters import ConsequentEstimator
from pyfume.SimpfulModelBuilder import SugenoFISBuilder
from pyfume.Tester import SugenoFISTester
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, cohen_kappa_score
from numpy import clip, column_stack, argmax, vectorize

In [3]:
# Import and clean the data
column_names = ["Clump_Thickness", "Uniformity_of_Cell_Size", "Uniformity_of_Cell_Shape", "Marginal_Adhesion", "Single_Epithelial_Cell_Size", "Bland_Chromatin", "Normal_Nucleoli", "Mitoses", "Something", "Cancer"]
data = pd.read_csv(os.path.abspath('wbco.csv'),names=column_names)
data.head()
rows_with_question_mark = data[data.isin(["?"]).any(axis=1)].index
data = data.drop(rows_with_question_mark)
data = data.astype('int')
data['Cancer'] = data['Cancer'].astype('int')
column_names = column_names[:-1]

# Set seed
seed = 7

# Normalize the dataset
scaler_input = MinMaxScaler()
scaler_output = MinMaxScaler()

# Choose predictors and output
X = scaler_input.fit_transform(data[column_names].values)
y = scaler_output.fit_transform(data["Cancer"].values.reshape(-1, 1))